In [1]:

outputdebug = False 
def debug(msg):
    if outputdebug:
        print(msg)

class Node():
    def __init__(self, key):
        self.key = key
        self.left = None 
        self.right = None 




class AVLTree():
    def __init__(self, *args):
        self.node = None 
        self.height = -1  
        self.balance = 0; 
        
        if len(args) == 1: 
            for i in args[0]: 
                self.insert(i)
                
    def height(self):
        if self.node: 
            return self.node.height 
        else: 
            return 0 
    
    def is_leaf(self):
        return (self.height == 0) 
    
    def insert(self, key):
        tree = self.node
        
        newnode = Node(key)
        
        if tree == None:
            self.node = newnode 
            self.node.left = AVLTree() 
            self.node.right = AVLTree()
            debug("Inserted key [" + str(key) + "]")
        
        elif key < tree.key: 
            self.node.left.insert(key)
            
        elif key > tree.key: 
            self.node.right.insert(key)
        
        else: 
            debug("Key [" + str(key) + "] already in tree.")
            
        self.rebalance() 
        
    def rebalance(self):
        ''' 
        Rebalance a particular (sub)tree
        ''' 
        # key inserted. Let's check if we're balanced
        self.update_heights(False)
        self.update_balances(False)
        while self.balance < -1 or self.balance > 1: 
            if self.balance > 1:
                if self.node.left.balance < 0:  
                    self.node.left.lrotate() # we're in case II
                    self.update_heights()
                    self.update_balances()
                self.rrotate()
                self.update_heights()
                self.update_balances()
                
            if self.balance < -1:
                if self.node.right.balance > 0:  
                    self.node.right.rrotate() # we're in case III
                    self.update_heights()
                    self.update_balances()
                self.lrotate()
                self.update_heights()
                self.update_balances()


            
    def rrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' right') 
        A = self.node 
        B = self.node.left.node 
        T = B.right.node 
        
        self.node = B 
        B.right.node = A 
        A.left.node = T 

    
    def lrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' left') 
        A = self.node 
        B = self.node.right.node 
        T = B.left.node 
        
        self.node = B 
        B.left.node = A 
        A.right.node = T 
        
            
    def update_heights(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_heights()
                if self.node.right != None:
                    self.node.right.update_heights()
            
            self.height = max(self.node.left.height,
                              self.node.right.height) + 1 
        else: 
            self.height = -1 
            
    def update_balances(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_balances()
                if self.node.right != None:
                    self.node.right.update_balances()

            self.balance = self.node.left.height - self.node.right.height 
        else: 
            self.balance = 0 

    def delete(self, key):
        # debug("Trying to delete at node: " + str(self.node.key))
        if self.node != None: 
            if self.node.key == key: 
                debug("Deleting ... " + str(key))  
                if self.node.left.node == None and self.node.right.node == None:
                    self.node = None # leaves can be killed at will 
                # if only one subtree, take that 
                elif self.node.left.node == None: 
                    self.node = self.node.right.node
                elif self.node.right.node == None: 
                    self.node = self.node.left.node
                
                # worst-case: both children present. Find logical successor
                else:  
                    replacement = self.logical_successor(self.node)
                    if replacement != None: # sanity check 
                        debug("Found replacement for " + str(key) + " -> " + str(replacement.key))  
                        self.node.key = replacement.key 
                        
                        # replaced. Now delete the key from right child 
                        self.node.right.delete(replacement.key)
                    
                self.rebalance()
                return  
            elif key < self.node.key: 
                self.node.left.delete(key)  
            elif key > self.node.key: 
                self.node.right.delete(key)
                        
            self.rebalance()
        else: 
            return 

    def logical_predecessor(self, node):
        ''' 
        Find the biggest valued node in LEFT child
        ''' 
        node = node.left.node 
        if node != None: 
            while node.right != None:
                if node.right.node == None: 
                    return node 
                else: 
                    node = node.right.node  
        return node 
    
    def logical_successor(self, node):
        ''' 
        Find the smallese valued node in RIGHT child
        ''' 
        node = node.right.node  
        if node != None: # just a sanity check  
            
            while node.left != None:
                debug("LS: traversing: " + str(node.key))
                if node.left.node == None: 
                    return node 
                else: 
                    node = node.left.node  
        return node 

    def check_balanced(self):
        if self == None or self.node == None: 
            return True
        
        # We always need to make sure we are balanced 
        self.update_heights()
        self.update_balances()
        return ((abs(self.balance) < 2) and self.node.left.check_balanced() and self.node.right.check_balanced())  
        
    def inorder_traverse(self):
        if self.node == None:
            return [] 
        
        inlist = [] 
        l = self.node.left.inorder_traverse()
        for i in l: 
            inlist.append(i) 

        inlist.append(self.node.key)

        l = self.node.right.inorder_traverse()
        for i in l: 
            inlist.append(i) 
    
        return inlist 

    def display(self, level=0, pref=''):
        '''
        Display the whole tree. Uses recursive def.
        TODO: create a better display using breadth-first search
        '''        
        self.update_heights()  # Must update heights before balances 
        self.update_balances()
        if(self.node != None): 
            print('-' * level * 2, pref, self.node.key, "[" + str(self.height) + ":" + str(self.balance) + "]", 'L' if self.is_leaf() else ' ')   
            if self.node.left != None: 
                self.node.left.display(level + 1, '<')
            if self.node.left != None:
                self.node.right.display(level + 1, '>')
        



In [2]:
a = AVLTree()
for i in [5,6,9,8,10,7,4,1,3,2]:
    a.insert(i)
a.height

3

In [3]:
import time
import numpy as np

t = AVLTree()

NUM = 1000

for i in range(10):
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.031239748001098633
1 번째 트리 높이:  11
1 - 1 번째 탐색, 찾는 숫자 11681 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 66119 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 76740 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 64468 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 77705 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.031256914138793945
2 번째 트리 높이:  12
2 - 1 번째 탐색, 찾는 숫자 56762 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 83724 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 27591 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 53628 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 97382 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.031239032745361328
3 번째 트리 높이:  13
3 - 1 번째 탐색, 찾는 숫자 50742 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 72362 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 55926 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 37766 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 5397 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.046823978424072266
4 번째 트리 높이:  13
4 - 1 번째 탐색, 찾는 숫자 86069 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 34488 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 63675 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 51999 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 49308 탐색+삭제 시간: 0.0

5 번째 

In [4]:
import time
import numpy as np

t = AVLTree()

NUM = 10000

for i in range(10):
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  0.3258833885192871
1 번째 트리 높이:  15
1 - 1 번째 탐색, 찾는 숫자 32752 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 29403 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 46384 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 60224 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 44233 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.41547155380249023
2 번째 트리 높이:  16
2 - 1 번째 탐색, 찾는 숫자 1855 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 50002 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 34780 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 10827 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 14724 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.4820375442504883
3 번째 트리 높이:  17
3 - 1 번째 탐색, 찾는 숫자 48473 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 37394 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 17365 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 10911 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 10699 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.42020583152770996
4 번째 트리 높이:  17
4 - 1 번째 탐색, 찾는 숫자 2481 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 89107 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 2759 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 35320 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 3702 탐색+삭제 시간: 0.0

5 번째 트리 생성 시간:

In [5]:
import time
import numpy as np

t = AVLTree()

NUM = 100000

for i in range(10):
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  4.317710638046265
1 번째 트리 높이:  18
1 - 1 번째 탐색, 찾는 숫자 3982 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 37849 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 86739 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 86364 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 13604 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  3.855994701385498
2 번째 트리 높이:  19
2 - 1 번째 탐색, 찾는 숫자 29060 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 5080 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 79360 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 94888 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 40996 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  3.268364429473877
3 번째 트리 높이:  19
3 - 1 번째 탐색, 찾는 숫자 44804 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 29782 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 1511 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 27198 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 48755 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  4.064647912979126
4 번째 트리 높이:  19
4 - 1 번째 탐색, 찾는 숫자 59496 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 46218 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 13033 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 55251 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 57362 탐색+삭제 시간: 0.0

5 번째 트리 생성 시간:  3.0

In [8]:
import time
import numpy as np

t = AVLTree()

NUM = 1000000

for i in range(10):
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

1 번째 트리 생성 시간:  32.91643190383911
1 번째 트리 높이:  19
1 - 1 번째 탐색, 찾는 숫자 76251 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 18445 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 43891 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 58711 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 87553 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  29.66185712814331
2 번째 트리 높이:  19
2 - 1 번째 탐색, 찾는 숫자 73937 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 15349 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 78570 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 43223 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 55409 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  31.681108713150024
3 번째 트리 높이:  19
3 - 1 번째 탐색, 찾는 숫자 19615 탐색+삭제 시간: 0.0009548664093017578
3 - 2 번째 탐색, 찾는 숫자 5101 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 85654 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 87858 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 11330 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  31.71998405456543
4 번째 트리 높이:  19
4 - 1 번째 탐색, 찾는 숫자 91858 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 53997 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 88071 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 82672 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자 11131 탐색+삭제 시간: 0.0